In [1]:
# input = """###############
# #...#...#.....#
# #.#.#.#.#.###.#
# #S#...#.#.#...#
# #######.#.#.###
# #######.#.#...#
# #######.#.###.#
# ###..E#...#...#
# ###.#######.###
# #...###...#...#
# #.#####.#.###.#
# #.#...#.#.#...#
# #.#.#.#.#.#.###
# #...#...#...###
# ###############"""

input = open("inputs/20").read()

In [2]:
import numpy as np


def parse_board(input):
    return np.array(list(map(list, input.splitlines())))


def add_tuple(t1, t2):
    return tuple(x + y for x, y in zip(t1, t2))


def print_board(board):
    print("\n".join("".join(row) for row in board))


dirs = {
    "^": (-1, 0),
    ">": (0, 1),
    "v": (1, 0),
    "<": (0, -1),
}

In [3]:
board = parse_board(input)

In [4]:
start = tuple(int(i) for i in np.argwhere(board == "S")[0])
end = tuple(int(i) for i in np.argwhere(board == "E")[0])

In [5]:
board[start] = "."
board[end] = "."

In [6]:
board

array([['#', '#', '#', ..., '#', '#', '#'],
       ['#', '.', '.', ..., '.', '.', '#'],
       ['#', '.', '#', ..., '#', '.', '#'],
       ...,
       ['#', '#', '#', ..., '#', '.', '#'],
       ['#', '#', '#', ..., '.', '.', '#'],
       ['#', '#', '#', ..., '#', '#', '#']], dtype='<U1')

In [7]:
import heapq
from collections import defaultdict

dirs = {
    "^": (-1, 0),
    ">": (0, 1),
    "v": (1, 0),
    "<": (0, -1),
}


def in_bounds(pos):
    return all(0 <= pos[i] < board.shape[i] for i in range(len(board.shape)))


def add_tuples(tuple1, tuple2):
    return tuple(map(lambda x, y: x + y, tuple1, tuple2))


In [8]:
def find_pattern_middle_indices(arr):
    """
    Finds the middle indices of a given linear pattern in a 2D array.

    Parameters:
        arr (list of list of str): The 2D array of strings.

    Returns:
        dict: A dictionary with two keys, 'horizontal' and 'vertical', containing lists of
              (row, col) tuples for the middle element of each pattern occurrence.
    """
    rows = len(arr)
    cols = len(arr[0]) if rows > 0 else 0
    p0, p1, p2 = ".#."

    horizontal_matches = []
    vertical_matches = []

    for i in range(rows):
        for j in range(cols - 2):
            if arr[i][j] == p0 and arr[i][j + 1] == p1 and arr[i][j + 2] == p2:
                horizontal_matches.append((i, j + 1))  # Middle element index

    for i in range(rows - 2):
        for j in range(cols):
            if arr[i][j] == p0 and arr[i + 1][j] == p1 and arr[i + 2][j] == p2:
                vertical_matches.append((i + 1, j))

    return {"horizontal": horizontal_matches, "vertical": vertical_matches}

In [9]:
clip_spots = find_pattern_middle_indices(board)

In [10]:
len(clip_spots["horizontal"]), len(clip_spots["vertical"])

(3562, 3404)

In [11]:
def solve_with_cheats(cheat_start, cheat_end):
    def get_possible_neighbors(pos):
        for dir in dirs.values():
            new_pos = add_tuples(pos, dir)
            cheat_allowed = (pos == cheat_start) and (new_pos == cheat_end)
            if (in_bounds(new_pos) and board[new_pos] != "#") or cheat_allowed:
                yield new_pos

    pq = [(start, 0)]
    distances = defaultdict(lambda: np.inf)
    distances[start] = 0
    heapq.heapify(pq)

    while pq:
        current_node, current_distance = heapq.heappop(pq)

        # if we've already found a better path skip
        # this relates to the inability to update the priorities in the heapq
        if distances[current_node] < current_distance:
            continue

        for n in get_possible_neighbors(current_node):
            if current_distance + 1 < distances[n]:
                distances[n] = current_distance + 1
                heapq.heappush(pq, (n, current_distance + 1))

    return distances[end]

In [12]:
normal_distance = solve_with_cheats((-1, -1), (-1, -1))
normal_distance

9464

In [13]:
from tqdm import tqdm

horizontal_dirs = [dirs["<"], dirs[">"]]
vertical_dirs = [dirs["^"], dirs["v"]]

improvements = []

for clip_spot in tqdm(clip_spots["horizontal"]):
    for cheat_end in [add_tuple(clip_spot, d) for d in horizontal_dirs]:
        dst = solve_with_cheats(cheat_end, clip_spot)
        improvements.append(normal_distance - dst)

for clip_spot in tqdm(clip_spots["vertical"]):
    for cheat_end in [add_tuple(clip_spot, d) for d in vertical_dirs]:
        dst = solve_with_cheats(cheat_end, clip_spot)
        improvements.append(normal_distance - dst)

100%|██████████| 3404/3404 [04:17<00:00, 13.21it/s]


In [14]:
improvements = np.array(improvements)

In [15]:
np.sort(improvements)

array([   0,    0,    0, ..., 9400, 9402, 9404])

In [16]:
(improvements >= 100).sum()

np.int64(1409)

In [17]:
(board == ".").sum()

np.int64(9465)

In [18]:
def where_to_tuples(mask):
    return [tuple(map(int, t)) for t in zip(*np.where(mask))]


def manhattan_distance(point1, point2):
    return abs(point1[0] - point2[0]) + abs(point1[1] - point2[1])


track_locs = where_to_tuples(board == ".")

In [19]:
len(track_locs)

9465

In [20]:
from itertools import combinations

MAX_CLIP = 20

blah = []
for pos1, pos2 in combinations(track_locs, 2):
    md = manhattan_distance(pos1, pos2)

    if md <= MAX_CLIP:
        blah.append((pos1, pos2))

In [21]:
len(blah)

1738089

In [22]:
def solve_from_start(start):
    def get_possible_neighbors(pos):
        for dir in dirs.values():
            new_pos = add_tuples(pos, dir)
            if in_bounds(new_pos) and board[new_pos] != "#":
                yield new_pos

    pq = [(start, 0)]
    distances = defaultdict(lambda: np.inf)
    distances[start] = 0
    heapq.heapify(pq)

    while pq:
        current_node, current_distance = heapq.heappop(pq)

        # if we've already found a better path skip
        # this relates to the inability to update the priorities in the heapq
        if distances[current_node] < current_distance:
            continue

        for n in get_possible_neighbors(current_node):
            if current_distance + 1 < distances[n]:
                distances[n] = current_distance + 1
                heapq.heappush(pq, (n, current_distance + 1))

    return distances

In [23]:
all_pairs_shortest = {}

for track in tqdm(track_locs):
    all_pairs_shortest[track] = solve_from_start(track)

100%|██████████| 9465/9465 [06:05<00:00, 25.90it/s]


In [24]:
all_pairs_shortest[start][end]

9464

In [25]:
from itertools import combinations

improvements = []
MAX_CLIP = 20
normal_distance = all_pairs_shortest[start][end]

for pos1, pos2 in combinations(track_locs, 2):
    md = manhattan_distance(pos1, pos2)

    if md > MAX_CLIP:
        continue

    d1 = all_pairs_shortest[start][pos1] + md + all_pairs_shortest[pos2][end]
    d2 = all_pairs_shortest[start][pos2] + md + all_pairs_shortest[pos1][end]

    improvements.append(normal_distance - d1)
    improvements.append(normal_distance - d2)

In [26]:
improvements = np.array(improvements)

In [27]:
(improvements >= 100).sum()

np.int64(1012821)

In [28]:
# I could redo pt1 with this pt2 solution but I'm too lazy for now